In [1]:
import random, dlib, cv2, os
import pandas as pd
import numpy as np
import face_recognition

import glob, os

훈련 및 테스트 데이터 만들기(img, landmark)

img

In [6]:
csv_files = glob.glob('./train/*.csv')
img_files = glob.glob('./train/dog/*.jpg')

img_size = 512

dataset = {
    'image' : [],
    'lmks' : [],
    'bbs' : []
}

for f in range(0, len(csv_files)):
            
    pd_frame = pd.read_csv(csv_files[f], sep = ',', header = None)
#     print(pd_frame.shape)
    
    landmarks = (pd_frame.iloc[0, 1:].values).reshape((-1, 2))
#     print(landmarks.shape)
#     print(landmarks)

    img = cv2.imread(img_files[f])
#     print(img)

    bb = np.array([np.min(landmarks, axis = 0), np.max(landmarks, axis = 0)]).astype(np.int32)
    
    dataset['image'].append(img)
    dataset['lmks'].append(landmarks.flatten())
    dataset['bbs'].append(bb.flatten())
    
np.save('C:/Users/YGL/Desktop/vision/세미/csv/img_train.npy', np.array(dataset))

In [9]:
csv_test_files = glob.glob('./test/*.csv')
img_test_files = glob.glob('./test/dog/*.jpg')

img_size = 512

dataset = {
    'image' : [],
    'lmks' : [],
    'bbs' : []
}

for f in range(0, len(csv_files)):
            
    pd_frame = pd.read_csv(csv_files[f], sep = ',', header = None)
#     print(pd_frame.shape)
    
    landmarks = (pd_frame.iloc[0, 1:].values).reshape((-1, 2))
#     print(landmarks.shape)
#     print(landmarks)

    img = cv2.imread(img_files[f])
#     print(img)

    bb = np.array([np.min(landmarks, axis = 0), np.max(landmarks, axis = 0)]).astype(np.int32)
    
    dataset['image'].append(img)
    dataset['lmks'].append(landmarks.flatten())
    dataset['bbs'].append(bb.flatten())
    
np.save('C:/Users/YGL/Desktop/vision/세미/csv/img_test.npy', np.array(dataset))

landmark

In [ ]:
csv_test_files = glob.glob('./test/*.csv')
img_test_files = glob.glob('./test/dog/*.jpg')

In [8]:
csv_files = glob.glob('./train/*.csv')
img_files = glob.glob('./train/dog/*.jpg')

img_size = 512

dataset = {
    'image' : [],
    'lmks' : [],
    'bbs' : []
}


for f in range(0, len(csv_files)):
            
    pd_frame = pd.read_csv(csv_files[f], sep = ',', header = None)
#     print(pd_frame.shape)
    
    landmarks = (pd_frame.iloc[0, 1:].values).reshape((-1, 2))
#     print(landmarks.shape)
#     print(landmarks)

    bb = np.array([np.min(landmarks, axis = 0), np.max(landmarks, axis = 0)]).astype(np.int32)
    center = np.mean(bb, axis = 0)
    
    face_size = max(np.abs(np.max(landmarks, axis = 0) - np.min(landmarks, axis = 0)))
    
    new_bb = np.array([
        center - face_size * 0.6,
        center + face_size * 0.6
    ]).astype(np.int32)
    
    new_bb = np.clip(new_bb, 0, 99999)
    new_landmarks = landmarks - new_bb[0]
    
    img = cv2.imread(img_files[f])
#     print(img)
    
    new_img = img[new_bb[0][1]:new_bb[1][1], new_bb[0][0]:new_bb[1][0]]
    
    dataset['image'].append(img)
    dataset['lmks'].append(new_landmarks.flatten())
    dataset['bbs'].append(new_bb.flatten())
    
np.save('C:/Users/YGL/Desktop/vision/세미/csv/lmks_train.npy', np.array(dataset))

In [10]:
csv_test_files = glob.glob('./test/*.csv')
img_test_files = glob.glob('./test/dog/*.jpg')
img_size = 512

dataset = {
    'image' : [],
    'lmks' : [],
    'bbs' : []
}


for f in range(0, len(csv_files)):
            
    pd_frame = pd.read_csv(csv_files[f], sep = ',', header = None)
#     print(pd_frame.shape)
    
    landmarks = (pd_frame.iloc[0, 1:].values).reshape((-1, 2))
#     print(landmarks.shape)
#     print(landmarks)

    bb = np.array([np.min(landmarks, axis = 0), np.max(landmarks, axis = 0)]).astype(np.int32)
    center = np.mean(bb, axis = 0)
    
    face_size = max(np.abs(np.max(landmarks, axis = 0) - np.min(landmarks, axis = 0)))
    
    new_bb = np.array([
        center - face_size * 0.6,
        center + face_size * 0.6
    ]).astype(np.int32)
    
    new_bb = np.clip(new_bb, 0, 99999)
    new_landmarks = landmarks - new_bb[0]
    
    img = cv2.imread(img_files[f])
#     print(img)
    
    new_img = img[new_bb[0][1]:new_bb[1][1], new_bb[0][0]:new_bb[1][0]]
    
    dataset['image'].append(img)
    dataset['lmks'].append(new_landmarks.flatten())
    dataset['bbs'].append(new_bb.flatten())
    
np.save('C:/Users/YGL/Desktop/vision/세미/csv/lmks_test.npy', np.array(dataset))

훈련하기

In [8]:
import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.keras.applications import mobilenet_v2

In [13]:
img_size = 512

mode = 'bbs'

if mode is 'bbs':
    output_size = 4
elif mode is 'lmks':
    output_size = 12
    
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

data_0 = np.load('./csv/img_train.npy', allow_pickle=True)
data_1 = np.load('./csv/lmks_train.npy', allow_pickle=True)
data_2 = np.load('./csv/img_test.npy', allow_pickle=True)
data_3 = np.load('./csv/lmks_test.npy', allow_pickle=True)

x_train = np.concatenate((data_0.item().get('image')))
y_train = np.concatenate((data_0.item().get(mode)))
                         
x_test = np.array(data_2.item().get('image'))
y_test = np.array(data_2.item().get(mode))

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (-1, img_size, img_size, 3))
x_test = np.reshape(x_test, (-1, img_size, img_size, 3))

y_train = np.reshape(y_train, (-1, output_size))
y_test = np.reshape(y_test, (-1, output_size))

inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3))
                    
mobilenetv2_model = mobilenet_v2.MobileNetV2(input_shape=(img_size, img_size, 3), alpha=1.0, include_top=False, weights='imagenet', input_tensor=inputs, pooling='max')

net = tf.keras.layers.Dense(128, activation='relu')(mobilenetv2_model.layers[-1].output)
net = tf.keras.layers.Dense(64, activation='relu')(net)
net = tf.keras.layers.Dense(output_size, activation='linear')(net)

model = tf.keras.models.Model(inputs=inputs, outputs=net)



model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [16]:
model.compile(optimizer='Adam', loss='mse', metrics = 'accuracy')

In [17]:
modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = keras.callbacks.ModelCheckpoint(filepath=modelpath,
                                            monitor='val_loss',
                                            verbose=1,
                                            save_best_only=True)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=10)

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50,
                   batch_size=30, verbose=1, shuffle = True,
                   callbacks=[checkpoint, early_stopping])

Epoch 1/50
17/17 [==============================] - ETA: 0s - loss: 8850.7139 - accuracy: 0.4594 
Epoch 00001: val_loss improved from inf to 1844.92102, saving model to ./model\01-1844.9210.hdf5
17/17 [==============================] - 369s 22s/step - loss: 8850.7139 - accuracy: 0.4594 - val_loss: 1844.9210 - val_accuracy: 0.6436
Epoch 2/50
17/17 [==============================] - ETA: 0s - loss: 1250.5690 - accuracy: 0.5683 
Epoch 00002: val_loss improved from 1844.92102 to 1674.08350, saving model to ./model\02-1674.0835.hdf5
17/17 [==============================] - 375s 22s/step - loss: 1250.5690 - accuracy: 0.5683 - val_loss: 1674.0835 - val_accuracy: 0.6059
Epoch 3/50
17/17 [==============================] - ETA: 0s - loss: 1008.5359 - accuracy: 0.6079 
Epoch 00003: val_loss did not improve from 1674.08350
17/17 [==============================] - 375s 22s/step - loss: 1008.5359 - accuracy: 0.6079 - val_loss: 1839.1985 - val_accuracy: 0.6436
Epoch 4/50
17/17 [======================

KeyboardInterrupt: 

In [ ]:
img_size = 512

mode = 'lmks'

if mode is 'bbs':
    output_size = 4
elif mode is 'lmks':
    output_size = 12
    
start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

data_0 = np.load('./csv/img_train.npy', allow_pickle=True)
data_1 = np.load('./csv/lmks_train.npy', allow_pickle=True)
data_2 = np.load('./csv/img_test.npy', allow_pickle=True)
data_3 = np.load('./csv/lmks_test.npy', allow_pickle=True)

x_train = np.concatenate((data_1.item().get('image')))
y_train = np.concatenate((data_1.item().get(mode)))
                         
x_test = np.array(data_3.item().get('image'))
y_test = np.array(data_3.item().get(mode))

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (-1, img_size, img_size, 3))
x_test = np.reshape(x_test, (-1, img_size, img_size, 3))

y_train = np.reshape(y_train, (-1, output_size))
y_test = np.reshape(y_test, (-1, output_size))

inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3))
                         
mobilenetv2_model = mobilenet_v2.MobileNetV2(input_shape=(img_size, img_size, 3), alpha=1.0, include_top=False, weights='imagenet', input_tensor=inputs, pooling='max')

net = tf.keras.layers.Dense(128, activation='relu')(mobilenetv2_model.layers[-1].output)
net = tf.keras.layers.Dense(64, activation='relu')(net)
net = tf.keras.layers.Dense(output_size, activation='linear')(net)

model = tf.keras.models.Model(inputs=inputs, outputs=net)

model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='mse', metrics = 'accuracy')

In [ ]:
modelpath = './land_model/{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = keras.callbacks.ModelCheckpoint(filepath=modelpath,
                                            monitor='val_loss',
                                            verbose=1,
                                            save_best_only=True)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=10)

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50,
                   batch_size=30, verbose=1, shuffle = True,
                   callbacks=[checkpoint, early_stopping])